In [ ]:
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime
import hist
import mplhep as hep
hep.style.use('CMS')

In [ ]:
con = sqlite3.connect('./desy_feb_2024/FullBaselineHistory.sqlite')
df = pd.read_sql_query("SELECT * from baselines", con)

In [ ]:
interest_df = df.loc[(df['col']==11) & (df['row']==8)].reset_index(drop=True)
interest_df

### Run info

In [ ]:
run_info = {
    ### Offset 20
    'Run32':{
        'start': datetime.datetime(2024, 2, 28, 3, 2),
        'duration': 5,
    },
    'Run33':{
        'start': datetime.datetime(2024, 2, 28, 8, 28),
        'duration': 5,
    },
    'Run34':{
        'start': datetime.datetime(2024, 2, 28, 13, 55),
        'duration': 4,
    },
    'Run35':{
        'start': datetime.datetime(2024, 2, 28, 18, 40),
        'duration': 4,
    },
    'Run36':{
        'start': datetime.datetime(2024, 2, 28, 22, 57),
        'duration': 5,
    },
    'Run37':{
        'start': datetime.datetime(2024, 2, 29, 4, 18),
        'duration': 5,
    },
    'Run38':{
        'start': datetime.datetime(2024, 2, 29, 9, 42),
        'duration': 5,
    },
    'Run39':{
        'start': datetime.datetime(2024, 2, 29, 15, 35),
        'duration': 2,
    },
    'Run40':{
        'start': datetime.datetime(2024, 2, 29, 20, 46),
        'duration': 5,
    },
    'Run41':{
        'start': datetime.datetime(2024, 3, 1, 2, 4),
        'duration': 5,
    },
    'Run42':{
        'start': datetime.datetime(2024, 3, 1, 7, 27),
        'duration': 5,
    },
    'Run43':{
        'start': datetime.datetime(2024, 3, 1, 12, 10),
        'duration': 5,
    },
    'Run44':{
        'start': datetime.datetime(2024, 3, 1, 17, 30),
        'duration': 5,
    },
    'Run45':{
        'start': datetime.datetime(2024, 3, 1, 23, 7),
        'duration': 5,
    },
    'Run46':{
        'start': datetime.datetime(2024, 3, 2, 4, 26),
        'duration': 5,
    },
    'Run47':{
        'start': datetime.datetime(2024, 3, 2, 9, 46),
        'duration': 5,
    },
    'Run48':{
        'start': datetime.datetime(2024, 3, 2, 15, 56),
        'duration': 4,
    },
}

### Select data

In [ ]:
interest_df['timestamp'] = pd.to_datetime(interest_df['timestamp'])

selected_baselines = {}

for key, val in run_info.items():
    cut_start_time = val['start'] -  datetime.timedelta(hours=1)
    cut_end_time = cut_start_time + datetime.timedelta(hours=val['duration'])
    filtered_df = interest_df.loc[(interest_df['timestamp'] > cut_start_time) & (interest_df['timestamp'] < cut_end_time)]
    selected_baselines[key] = filtered_df['baseline'].astype('int32').values

# interest_df['timestamp'] = pd.to_datetime(interest_df['timestamp'])
# interest_df = interest_df.loc[(interest_df['timestamp'] > run_info['Run32']['start'] -  datetime.timedelta(hours=1))]

In [ ]:
selected_baselines

In [ ]:
for k in range(4):
    x_labels = list(selected_baselines.keys())
    x_range = np.arange(0, len(x_labels))
    y_vals = []
    for key, val in selected_baselines.items():
        y_vals.append(val[k])

    fig, ax = plt.subplots(figsize=(11, 11))
    ax.plot(x_range, y_vals, marker='o')
    ax.set_title(df['chip_name'].unique()[k], loc='right', fontsize=18)
    ax.set_xticks(x_range)
    ax.set_xticklabels(x_labels, rotation=90)
    ax.yaxis.set_major_locator(plt.MaxNLocator(integer=True)) # Setting y-ticks to display only integers
    plt.minorticks_off()
    ax.grid()
    plt.tight_layout()

### Baseline over time

In [ ]:
draw_dates = True

fig, ax = plt.subplots(figsize=(20, 10))
hep.cms.text(loc=0, ax=ax, text="Phase-2 Preliminary", fontsize=25)
ax.set_title('Baseline of pixel (8, 11)', size=17, loc="right")

for idx, iboard in enumerate(interest_df['chip_name'].unique()):
    if idx == 0: continue
    tmp_df = interest_df.loc[interest_df['chip_name']==iboard]
    tmp_df['timestamp'] = pd.to_datetime(tmp_df['timestamp'])
    ax.plot(tmp_df['timestamp'], tmp_df['baseline'], label=f'{iboard}', marker='.')

    if idx == 0:
        date_form = mdates.DateFormatter("%m-%d\n%H:%M")
        ax.xaxis.set_major_formatter(date_form)

if draw_dates:
    for irun, itime in run_info.items():
        start_time = itime['start']
        end_time = start_time + datetime.timedelta(hours=itime['duration'])
        text_start_time = start_time - datetime.timedelta(hours=1)
        text_end_time = end_time + datetime.timedelta(hours=1)
        ax.vlines(x=pd.to_datetime(start_time), ymin=430, ymax=465, colors='magenta', linestyles='dashed')
        # ax.text(x=pd.to_datetime(text_start_time), y=465, s=f'{irun} Start', rotation=90, va='center', fontsize=12, color='magenta')
        ax.vlines(x=pd.to_datetime(end_time), ymin=430, ymax=465, colors='black', linestyles='dashdot')
        # ax.text(x=pd.to_datetime(text_end_time), y=465, s=f'{irun} End', rotation=90, va='center', fontsize=12)

ax.legend(loc='center left')

### Baseline in 1D hist

In [ ]:
for idx, iboard in enumerate(interest_df['chip_name'].unique()):
    fig, ax = plt.subplots(figsize=(10, 10))
    hep.cms.text(loc=0, ax=ax, text="Phase-2 Preliminary", fontsize=25)
    tmp_df = interest_df.loc[interest_df['chip_name']==iboard]
    most_val = tmp_df['baseline'].mode()[0]
    h = hist.Hist(hist.axis.Regular(20, most_val-10, most_val+10, name='test', label=f'{iboard}'))
    h.fill(tmp_df['baseline'])
    yerr_bool = np.array(h.values(), dtype=bool)
    hep.histplot(h, ax=ax, yerr=yerr_bool)

### Temperature

In [ ]:
con = sqlite3.connect('./ADCHistory_start.sqlite')
adc_df = pd.read_sql_query("SELECT * from adc", con)

In [ ]:
# names = ['ET2_EPIR_Pair1', 'ET2p01_BAR_4', 'ET2p01_BAR_5', 'ET2_EPIR_Pair4', 'Ambient Temp']

fig, ax = plt.subplots(figsize=(20, 10))
hep.cms.text(loc=0, ax=ax, text="Phase-2 Preliminary", fontsize=25)
ax.set_title('Temperature', size=17, loc="right")

for idx, iboard in enumerate(sorted(adc_df['channel'].unique())):
    tmp_df = adc_df.loc[adc_df['channel']==iboard]
    tmp_df['timestamp'] = pd.to_datetime(tmp_df['timestamp'])
    ax.plot(tmp_df['timestamp'], tmp_df['voltage'])#, label=f'{names[idx]}')

    if idx == 0:
        date_form = mdates.DateFormatter("%m-%d\n%H:%M")
        ax.xaxis.set_major_formatter(date_form)

# for irun, itime in run_info.items():
#     start_time = itime['start']
#     end_time = start_time + datetime.timedelta(hours=itime['duration'])
#     text_start_time = start_time - datetime.timedelta(hours=3)
#     text_end_time = end_time + datetime.timedelta(hours=1)
#     ax.vlines(x=pd.to_datetime(start_time), ymin=0, ymax=1.5, colors='magenta', linestyles='dashed')
#     ax.text(x=pd.to_datetime(text_start_time), y=1.4, s=f'{irun} Start', rotation=90, va='center', fontsize=12, color='magenta')
#     ax.vlines(x=pd.to_datetime(end_time), ymin=0, ymax=1.5, colors='black', linestyles='dashdot')
#     ax.text(x=pd.to_datetime(text_end_time), y=1.4, s=f'{irun} End', rotation=90, va='center', fontsize=12)

ax.set_ylabel('Voltage')
ax.legend(loc='best')